In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.formula.api as sm
import numpy as np
import pingouin as pg

In [2]:
df = pd.read_csv("for_ml.csv")
df.set_index('date',inplace=True)
#df['h1n1_dum'] = 1 *(df['h1n1_deaths'] != 0)
#df['covid_dum'] = 1 *(df['covid_deaths'] != 0)

In [3]:
df.pcorr().round(3)

,us_pop,alcohol_deaths,alcohol_sales,drug_deaths,homicides,perc_unemp,suicide_deaths,h1n1_deaths,covid_deaths
us_pop,1.000,-0.350,0.170,0.022,-0.033,0.379,0.133,-0.178,0.212
alcohol_deaths,-0.350,1.000,0.671,0.307,-0.366,0.899,0.597,-0.692,-0.190
alcohol_sales,0.170,0.671,1.000,-0.912,0.934,-0.287,-0.995,1.000,0.071
drug_deaths,0.022,0.307,-0.912,1.000,0.997,0.131,-0.947,0.900,-0.007
homicides,-0.033,-0.366,0.934,0.997,1.000,-0.072,0.965,-0.924,-0.043
perc_unemp,0.379,0.899,-0.287,0.131,-0.072,1.000,-0.193,0.314,0.111
suicide_deaths,0.133,0.597,-0.995,-0.947,0.965,-0.193,1.000,0.992,0.074
h1n1_deaths,-0.178,-0.692,1.000,0.900,-0.924,0.314,0.992,1.000,-0.081
covid_deaths,0.212,-0.190,0.071,-0.007,-0.043,0.111,0.074,-0.081,1.000


In [4]:
df.corr().round(3)

,us_pop,alcohol_deaths,alcohol_sales,drug_deaths,homicides,perc_unemp,suicide_deaths,h1n1_deaths,covid_deaths
us_pop,1.000,0.872,0.856,0.912,0.814,0.384,0.857,0.003,0.280
alcohol_deaths,0.872,1.000,0.797,0.945,0.644,0.192,0.740,-0.061,0.576
alcohol_sales,0.856,0.797,1.000,0.823,0.719,0.295,0.755,-0.054,0.387
drug_deaths,0.912,0.945,0.823,1.000,0.715,0.252,0.799,-0.057,0.461
homicides,0.814,0.644,0.719,0.715,1.000,0.444,0.676,0.065,0.261
perc_unemp,0.384,0.192,0.295,0.252,0.444,1.000,0.209,0.282,0.190
suicide_deaths,0.857,0.740,0.755,0.799,0.676,0.209,1.000,-0.073,0.147
h1n1_deaths,0.003,-0.061,-0.054,-0.057,0.065,0.282,-0.073,1.000,-0.031
covid_deaths,0.280,0.576,0.387,0.461,0.261,0.190,0.147,-0.031,1.000


In [5]:
X=df[['alcohol_deaths','alcohol_sales','drug_deaths','homicides','h1n1_deaths','perc_unemp']]


y=df[['suicide_deaths']]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression()

In [7]:
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for alcohol_deaths is -0.07432222602728741
The coefficient for alcohol_sales is 0.04938790523010525
The coefficient for drug_deaths is 0.17690071195394608
The coefficient for homicides is 0.08638356643815501
The coefficient for h1n1_deaths is -0.004796184238336094
The coefficient for perc_unemp is -0.0020511651641447054


In [8]:
intercept = regression_model.intercept_[0]
print("The intercept for our model is {}".format(intercept))

The intercept for our model is 0.7065326139417767


In [9]:
print("y = ",intercept, "+")
for idx, col_name in enumerate(X_train.columns):
    print("{}({}) + ".format(col_name, round(regression_model.coef_[0][idx],2)))
#Our regression line for this model is: 

y =  0.7065326139417767 +
alcohol_deaths(-0.07) + 
alcohol_sales(0.05) + 
drug_deaths(0.18) + 
homicides(0.09) + 
h1n1_deaths(-0.0) + 
perc_unemp(-0.0) + 


In [10]:
regression_model.score(X_test, y_test)

0.5351132009484352

In [11]:
lm = sm.ols(formula='suicide_deaths ~ alcohol_deaths + alcohol_sales + drug_deaths + homicides + h1n1_deaths + perc_unemp', data=df)
fit = lm.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         suicide_deaths   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     88.81
Date:                Fri, 29 Jul 2022   Prob (F-statistic):           1.25e-58
Time:                        12:42:34   Log-Likelihood:                 301.63
No. Observations:                 252   AIC:                            -589.3
Df Residuals:                     245   BIC:                            -564.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.7507      0.034     22.070      0.000       0.684       0.818
alcohol_deaths    -0.1263      0.085     -1.495      0.136      -0.293       0.040
alcohol_sales      0.0469      0.012      3.812      0.000       0.023       0.071
drug_deaths        0.1775      0.035      5.106      0.000       0.109       0.246
homicides          0.1025      0.034      2.980      0.003       0.035       0.170
h1n1_deaths       -0.0055      0.008     -0.671      0.503      -0.022       0.011
perc_unemp        -0.0032      0.002     -1.567      0.118      -0.007       0.001
==============================================================================
Omnibus:                        2.279   Durbin-Watson:                   0.939
Prob(Omnibus):                  0.320   Jarque-Bera (JB):                2.284
Skew:                          -0.229   Prob(JB):                        0.319
Kurtosis:                       2.910   Cond. No.                         139.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# partial Correlation
pg.partial_corr(data=df, x='alcohol_deaths', y='suicide_deaths', covar=['alcohol_sales','drug_deaths','homicides','h1n1_deaths','perc_unemp'])

,n,r,CI95%,p-val
pearson,252,-0.095059,"[-0.22, 0.03]",0.136287


In [13]:
from sklearn.metrics import mean_squared_error

y_predict = regression_model.predict(X_test)

#y_test
#dates = y_test[y_test["su"]==1]
#print(dates)
#print(y_test)
#print(y_predict)

In [14]:
regression_model_mse = mean_squared_error(y_predict, y_test)
regression_model_mse

0.007899273651376024

In [15]:
#covid

In [16]:
X=df[['alcohol_deaths','alcohol_sales','drug_deaths','homicides','covid_deaths','perc_unemp']]


y=df[['suicide_deaths']]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression()

In [18]:
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for alcohol_deaths is 0.1485380845044878
The coefficient for alcohol_sales is 0.04480515728987717
The coefficient for drug_deaths is 0.14336785590035936
The coefficient for homicides is 0.06319369948404152
The coefficient for covid_deaths is -0.004265621406160526
The coefficient for perc_unemp is 0.0001690745428154219


In [19]:
intercept = regression_model.intercept_[0]
print("The intercept for our model is {}".format(intercept))

The intercept for our model is 0.596730502871037


In [20]:
print("y = ",intercept, "+")
for idx, col_name in enumerate(X_train.columns):
    print("{}({}) + ".format(col_name, round(regression_model.coef_[0][idx],2)))
#Our regression line for this model is: 

y =  0.596730502871037 +
alcohol_deaths(0.15) + 
alcohol_sales(0.04) + 
drug_deaths(0.14) + 
homicides(0.06) + 
covid_deaths(-0.0) + 
perc_unemp(0.0) + 


In [21]:
regression_model.score(X_test, y_test)

0.6642966985775908

In [22]:
lm = sm.ols(formula='suicide_deaths ~ alcohol_deaths + alcohol_sales + drug_deaths + homicides + covid_deaths + perc_unemp', data=df)
fit = lm.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         suicide_deaths   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     117.6
Date:                Fri, 29 Jul 2022   Prob (F-statistic):           3.19e-69
Time:                        12:42:34   Log-Likelihood:                 326.88
No. Observations:                 252   AIC:                            -639.8
Df Residuals:                     245   BIC:                            -615.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5988      0.037     16.199      0.000       0.526       0.672
alcohol_deaths     0.1950      0.088      2.216      0.028       0.022       0.368
alcohol_sales      0.0445      0.011      4.006      0.000       0.023       0.066
drug_deaths        0.1139      0.033      3.494      0.001       0.050       0.178
homicides          0.0754      0.031      2.411      0.017       0.014       0.137
covid_deaths      -0.0036      0.000     -7.410      0.000      -0.005      -0.003
perc_unemp        -0.0007      0.002     -0.366      0.715      -0.004       0.003
==============================================================================
Omnibus:                        5.428   Durbin-Watson:                   0.994
Prob(Omnibus):                  0.066   Jarque-Bera (JB):                5.285
Skew:                          -0.353   Prob(JB):                       0.0712
Kurtosis:                       3.063   Cond. No.                         272.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
from sklearn.metrics import mean_squared_error

y_predict = regression_model.predict(X_test)

#y_test
#dates = y_test[y_test["su"]==1]
#print(dates)
#print(y_test)
#print(y_predict)

In [24]:
regression_model_mse = mean_squared_error(y_predict, y_test)
regression_model_mse

0.005704210678849249